In [33]:
import networkx as nx
import pandas as pd

In [34]:
df = pd.read_csv(r'CrawlData\data\reprocessed_data\relationship.csv')
df.head()

,id1,id2
0,8674,2382
1,8674,12512
2,8674,10317
3,8674,1098
4,8674,14219


In [35]:
nodes = set(list(df['id1']) + list(df['id2']))

In [36]:
graph = nx.Graph()

In [37]:
graph.add_nodes_from(nodes)
for i, r in df.iterrows():
    graph.add_edge(r['id1'], r['id2'])

In [38]:
# plt.figure(figsize=(16,16))
# pos = nx.spring_layout(graph)
# nx.draw_networkx_labels(graph, pos, font_size=10)
# nx.draw_networkx_edges(graph, pos, alpha=0.2)
# nx.draw_networkx_nodes(graph, pos, node_size=3)

In [39]:
def friends(graph: nx.Graph, user):
    # return user's set of friends
    return set(graph.neighbors(user))

In [40]:
def number_of_friends(graph: nx.Graph, user):
    # return the number of user's friends
    return len(set(graph.neighbors(user)))

In [41]:
def friends_of_friends(graph: nx.Graph, user):
    # return a set of friends of user's friends
    user_friends = friends(graph, user)
    friends_of_friends_set = set()
    for u in user_friends:
        for uu in friends(graph, u):
            if uu not in user_friends:
                friends_of_friends_set.add(uu)
    try:
        friends_of_friends_set.remove(user)
    except:
        return friends_of_friends_set
    return friends_of_friends_set

In [42]:
def mutual_friends(graph: nx.Graph, user1, user2):
    # return 2 users' set of mutual friends
    user1_friends = friends(graph, user1)
    user2_friends = friends(graph, user2)
    return user1_friends.intersection(user2_friends)

In [43]:
def number_of_mutual_friends(graph: nx.Graph, user1, user2):
    # return the number of 2 users' mutual friends
    return len(mutual_friends(graph, user1, user2))

In [55]:
def number_of_mutual_friends_map(graph: nx.Graph, user):
    all = friends_of_friends(graph, user)
    list_map = []
    for u in all:
        list_map.append((u, number_of_mutual_friends(graph, user, u)))
    return list_map

In [56]:
def get_ranked_list_by_mutual_friends(graph: nx.Graph, user, k = 10):
    ranked_list = sorted(number_of_mutual_friends_map(graph, user), key=lambda x: x[1], reverse=True)
    return ranked_list[:k]

In [57]:
def influence_score(graph: nx.Graph, user1, user2):
    mutual_friends_set = mutual_friends(graph, user1, user2)
    score = 0
    for u in mutual_friends_set:
        score = score + 1/number_of_friends(graph, u)
    return score

In [67]:
def influence_score_map(graph: nx.Graph, user):
    all = friends_of_friends(graph, user)
    list_map = []
    for u in all:
        list_map.append((u, influence_score(graph, user, u)))
    return list_map

In [59]:
def get_ranked_list_by_influence_score(graph: nx.Graph, user, k = 10):
    ranked_list = sorted(influence_score_map(graph, user), key= lambda x: x[1], reverse=True)
    return ranked_list[:k]

In [60]:
def get_recommended_list(graph: nx.Graph, user, option: str, k = 10):
    if option == 'influence':
        return get_ranked_list_by_influence_score(graph, user, k=k)
    elif option == 'friends':
        return get_ranked_list_by_mutual_friends(graph, user, k=k)
    else: 
        print('parameter "option" accept "influence" or "friends"')
        return []

In [65]:
get_recommended_list(graph, 3591, 'friends', 20)

[(19960, 60),
 (20319, 57),
 (16685, 54),
 (19110, 45),
 (18489, 44),
 (3843, 42),
 (5739, 42),
 (15491, 42),
 (18586, 40),
 (13087, 39),
 (13311, 36),
 (15238, 36),
 (18771, 36),
 (3013, 35),
 (10014, 35),
 (14438, 35),
 (965, 34),
 (13389, 34),
 (6324, 33),
 (17655, 32)]

In [68]:
get_recommended_list(graph, 3591, 'influence', 20)

[(19960, 1.9258206200103503),
 (3013, 1.6073750111610532),
 (20319, 1.5176810121633673),
 (19110, 1.2902558624720355),
 (10170, 1.2586106633079148),
 (16685, 1.254234522587314),
 (1800, 1.219350890038944),
 (3843, 1.1916233646951384),
 (18159, 1.1597597975559406),
 (12891, 1.0280448790021572),
 (18771, 1.0240990121547386),
 (965, 1.0226622357727768),
 (5739, 1.0195724762769374),
 (18106, 0.933303709926113),
 (18586, 0.9149730365416389),
 (18489, 0.9054244834240247),
 (15491, 0.823931667937174),
 (13389, 0.8134395720454293),
 (14545, 0.7921988013150549),
 (10014, 0.7736546118507325)]